# LangGraph translation demo - Multi-Step Pipeline

This notebook demonstrates the enhanced **multi-step "generate-and-filter"** translation pipeline.

The new pipeline has 6 stages:
1. **analyse_sense** - Understand semantic nuances
2. **translate_definition** - Translate definition with context
3. **translate_all_lemmas** - Direct translation of each lemma
4. **expand_synonyms** - Broaden candidate pool in target language
5. **filter_synonyms** - Quality check to remove imperfect matches
6. **assemble_result** - Combine all outputs into final synset

This approach generates high-quality synsets (sets of synonymous literals) rather than a single "headword".

In [ ]:
from pathlib import Path
import json
from pprint import pprint

DATA_PATH = Path("../examples/serbian_english_synset_pairs_enhanced.json")
with DATA_PATH.open("r", encoding="utf-8") as f:
    dataset = json.load(f)

pairs = dataset["pairs"]
metadata = dataset.get("metadata", {})

print(f"Loaded {len(pairs)} pairs from {DATA_PATH}")
print("Metadata snapshot:")
pprint(metadata)

sample_pair = pairs[0]
print("\nFirst pair keys:", list(sample_pair.keys()))

In [ ]:
pprint({
    "english_id": sample_pair.get("english_id"),
    "english_lemmas": sample_pair.get("english_lemmas"),
    "english_definition": sample_pair.get("english_definition"),
    "english_examples": sample_pair.get("english_examples"),
    "serbian_synonyms": sample_pair.get("serbian_synonyms"),
    "serbian_definition": sample_pair.get("serbian_definition"),
    "serbian_usage": sample_pair.get("serbian_usage"),
})

In [ ]:
import importlib
import ollama
import wordnet_autotranslate.pipelines.langgraph_translation_pipeline as lg_module

lg_module = importlib.reload(lg_module)
LangGraphTranslationPipeline = lg_module.LangGraphTranslationPipeline

PREFERRED_OLLAMA_MODEL = "gpt-oss:120b"
OLLAMA_TIMEOUT = 180  # seconds
OLLAMA_TEMPERATURE = 0.0

try:
    model_list_response = ollama.list()
    available_models = {item.model for item in model_list_response.models}
except Exception as exc:  # pragma: no cover - depends on local runtime
    raise RuntimeError(
        "Could not reach the local Ollama daemon. Start it with `ollama serve`."
    ) from exc

if not available_models:
    raise RuntimeError(
        "No Ollama models are installed. Pull one with `ollama pull <model>` before running this cell."
    )

if PREFERRED_OLLAMA_MODEL in available_models:
    ollama_model = PREFERRED_OLLAMA_MODEL
else:
    ollama_model = sorted(available_models)[0]
    print(
        f"Preferred model '{PREFERRED_OLLAMA_MODEL}' not found. "
        f"Falling back to '{ollama_model}'."
    )

pipeline = LangGraphTranslationPipeline(
    source_lang="en",
    target_lang="sr",
    model=ollama_model,
    temperature=OLLAMA_TEMPERATURE,
    timeout=OLLAMA_TIMEOUT,
)

print(f"Using Ollama model: {ollama_model}")

In [ ]:
preview_limit = 200


def preview_text(text: str | None, limit: int = preview_limit) -> str:
    if not text:
        return ""
    return text[:limit] + ("… [truncated]" if len(text) > limit else "")


synset_input = {
    "id": sample_pair.get("english_id"),
    "english_id": sample_pair.get("english_id"),
    "lemmas": sample_pair.get("english_lemmas", []),
    "definition": sample_pair.get("english_definition", ""),
    "examples": sample_pair.get("english_examples", []),
    "pos": sample_pair.get("english_pos"),
}

print("=" * 70)
print("TRANSLATING SYNSET WITH MULTI-STEP PIPELINE")
print("=" * 70)
print(f"Input synset ID: {synset_input.get('id')}")
print(f"English lemmas: {synset_input.get('lemmas')}")
print(f"Definition: {synset_input.get('definition')}")
print("\nRunning through 7-stage pipeline...")
print("  Stage 1: analyse_sense")
print("  Stage 2: translate_definition")
print("  Stage 3: translate_all_lemmas (NEW)")
print("  Stage 4: expand_synonyms (NEW)")
print("  Stage 5: filter_synonyms (NEW)")
print("  Stage 6: review_definition_quality (NEW)")
print("  Stage 7: assemble_result")
print("=" * 70)

result = pipeline.translate_synset(synset_input)

translation = result.get("translation", "")
definition_translation = result.get("definition_translation", "")
translated_synonyms = result.get("translated_synonyms", [])
examples = result.get("examples", [])
notes = result.get("notes")
curator_summary = result.get("curator_summary", "")
raw_response = result.get("raw_response", "")

print("\n" + "=" * 70)
print("RESULTS")
print("=" * 70)
print(f"Representative literal: {translation}")
print(f"Final synset ({len(translated_synonyms)} literals): {translated_synonyms}")
print(f"Example count: {len(examples)}")
print(f"Definition translation length: {len(definition_translation)} characters")

print("\n" + "=" * 70)
print("CURATOR SUMMARY")
print("=" * 70)
print(curator_summary)

print("\n" + "=" * 70)
print("DEFINITION TRANSLATION")
print("=" * 70)
print(definition_translation)

# Show pipeline progression
print("\n" + "=" * 70)
print("PIPELINE STAGE DETAILS")
print("=" * 70)

payload = result.get("payload", {})
initial_payload = payload.get("initial_translation", {})
expansion_payload = payload.get("expansion", {})
filtering_payload = payload.get("filtering", {})
definition_quality_payload = payload.get("definition_quality", {}) or {}

initial_translations = initial_payload.get("initial_translations", [])
expanded_synonyms = expansion_payload.get("expanded_synonyms", [])
filtered_synonyms = filtering_payload.get("filtered_synonyms", [])

print(f"\n📊 Stage 3 - Initial Translations ({len(initial_translations)} lemmas):")
for i, trans in enumerate(initial_translations, 1):
    print(f"  {i}. {trans}")

print(f"\n🔍 Stage 4 - Expanded Candidates ({len(expanded_synonyms)} synonyms):")
for i, syn in enumerate(expanded_synonyms, 1):
    print(f"  {i}. {syn}")

print(f"\n✅ Stage 5 - Filtered Results ({len(filtered_synonyms)} final literals):")
for i, lit in enumerate(filtered_synonyms, 1):
    print(f"  {i}. {lit}")

# Show what was removed during filtering
removed = set(expanded_synonyms) - set(filtered_synonyms)
if removed:
    print(f"\n❌ Removed during filtering ({len(removed)} items):")
    for item in sorted(removed):
        print(f"  - {item}")
else:
    print("\n✓ No items removed during filtering (all candidates passed validation)")

definition_quality_status_raw = (definition_quality_payload.get("status") or "unknown").lower()
definition_quality_status = definition_quality_status_raw.replace("_", " ").title()
definition_quality_issues = definition_quality_payload.get("issues") or []
definition_quality_revision = definition_quality_payload.get("revised_definition")
definition_quality_notes = definition_quality_payload.get("notes")
definition_quality_revision_text = definition_quality_revision.strip() if isinstance(definition_quality_revision, str) else ""
definition_quality_revision_applied = bool(definition_quality_revision_text) and definition_quality_status_raw == "needs_revision"

print(f"\n🧪 Stage 6 - Definition Quality Review: {definition_quality_status}")
if definition_quality_issues:
    for issue in definition_quality_issues:
        issue_type = issue.get("type", "unknown")
        issue_message = issue.get("message", "")
        print(f"  - {issue_type}: {issue_message}")
else:
    print("  No issues flagged.")

if definition_quality_revision_text:
    print("  Revised definition candidate:")
    print(f"    {preview_text(definition_quality_revision_text, 200)}")

if definition_quality_notes:
    print(f"  Notes: {preview_text(definition_quality_notes, 200)}")

print("\n🏁 Stage 7 - assemble_result → final synset ready for export")

if examples:
    print("\n" + "=" * 70)
    print("EXAMPLES")
    print("=" * 70)
    for i, ex in enumerate(examples[:3], 1):
        print(f"{i}. {preview_text(ex)}")
    if len(examples) > 3:
        print(f"... ({len(examples)} total)")

if notes:
    print("\n" + "=" * 70)
    print("NOTES")
    print("=" * 70)
    print(preview_text(notes))

print("\n" + "=" * 70)
print("STAGE LOGS (Preview)")
print("=" * 70)
logs = result.get("payload", {}).get("logs", {})
for stage, log in logs.items():
    if not log:
        continue
    print(f"\n[{stage.upper()}]")
    print(f"  Prompt preview: {preview_text(log.get('prompt'), 150)}")
    print(f"  Response preview: {preview_text(log.get('raw_response_preview'), 150)}")

print("\n" + "=" * 70)
print("FULL RAW RESPONSE (Last Stage)")
print("=" * 70)
print(preview_text(raw_response, 400))

In [ ]:
# Extract final synset for simple JSON output
# The new pipeline produces a clean list of validated synonyms

simple_output = {
    "english_id": synset_input.get("id"),
    "representative_literal": translation,  # First literal (for convenience)
    "synset_literals": translated_synonyms,  # The actual synset
    "literal_count": len(translated_synonyms),
    "definition_translation": definition_translation,
    "definition_quality": {
        "status": definition_quality_status_raw,
        "issues": definition_quality_issues,
        "revision_applied": definition_quality_revision_applied,
    },
    "pipeline_stages": {
        "initial_translations": len(initial_translations),
        "expanded_candidates": len(expanded_synonyms),
        "filtered_results": len(filtered_synonyms),
        "definition_quality_review": definition_quality_status_raw,
        "removal_rate": f"{len(removed)}/{len(expanded_synonyms)}" if expanded_synonyms else "0/0"
    },
}

output_path = Path("simple_translation_output.json")
with output_path.open("w", encoding="utf-8") as fp:
    json.dump(simple_output, fp, ensure_ascii=False, indent=2)

print("=" * 70)
print("SIMPLE JSON OUTPUT")
print("=" * 70)
print(f"Saved to: {output_path.resolve()}\n")
print(json.dumps(simple_output, ensure_ascii=False, indent=2))

## Understanding the Multi-Step Pipeline

### Generate → Expand → Filter → Review Workflow

The pipeline now uses a **4-stage synonym workflow** before final assembly:

1. **Generate Initial** (`translate_all_lemmas`): Direct translation of each English lemma
2. **Expand** (`expand_synonyms`): Find additional synonyms in the target language
3. **Filter** (`filter_synonyms`): Remove candidates with imperfect sense alignment
4. **Review Definition** (`review_definition_quality`): Polish translated gloss for grammar and style

### Benefits

✅ **Higher Quality**: Four-stage validation ensures precision
✅ **Broader Coverage**: Expansion finds native synonyms, not just literal translations
✅ **Stylistic Assurance**: Definition review catches grammar and circularity issues
✅ **Traceability**: Full audit trail showing progression
✅ **No Headword**: Output is a true synset (set of synonymous literals)

### Next steps

- Translate multiple synsets: `pipeline.translate(list_of_synsets)`
- Use streaming for large batches: `pipeline.translate_stream(synsets)`
- Inspect `filtering_payload` to see which candidates were rejected
- Review `definition_quality` payload for grammar/style diagnostics
- Compare quality against the old single-step approach

## 🔍 Accessing Full LLM Outputs (Untruncated)

The pipeline preserves **two versions** of LLM call data:

- **`result["payload"]["logs"]`** - Truncated summaries for quick viewing (500 chars)
- **`result["payload"]["calls"]`** - **Full, untruncated** LLM interactions

The truncation exists to prevent memory bloat when processing hundreds of synsets, but you can always access the complete data via the `calls` dictionary.

In [ ]:
# Example: View truncated logs (quick summary)
print("=== TRUNCATED LOGS (for quick viewing) ===\n")
print("Filtering stage log (truncated):")
truncated_log = result["payload"]["logs"]["filtering"]
print(f"  Raw response preview: {truncated_log.get('raw_response_preview', 'N/A')[:100]}...")
print(f"  (Response truncated at {len(truncated_log.get('raw_response_preview', ''))} chars)")


In [ ]:
# Example: Access FULL untruncated data
print("\n=== FULL UNTRUNCATED CALL DATA ===\n")
print("Filtering stage call (complete):")
full_call = result["payload"]["calls"]["filtering"]
print(f"  Stage: {full_call['stage']}")
print(f"  Full raw response length: {len(full_call['raw_response'])} chars")
print(f"  Full raw response:\n{full_call['raw_response']}")
print(f"\n  Parsed payload: {full_call['payload']}")

In [ ]:
# Save full logs to file for later analysis
import json
from pathlib import Path

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

full_logs_path = output_dir / "full_llm_logs.json"

# Extract all full calls for detailed analysis
full_logs = {
    "synset_id": result["source"]["id"],
    "translation": result["translation"],
    "all_stages": {
        stage: {
            "prompt": call.get("prompt", ""),
            "raw_response": call.get("raw_response", ""),
            "parsed_payload": call.get("payload", {}),
            "messages": call.get("messages", [])
        }
        for stage, call in result["payload"]["calls"].items()
    }
}

with open(full_logs_path, "w", encoding="utf-8") as f:
    json.dump(full_logs, f, indent=2, ensure_ascii=False)

print(f"✅ Full untruncated logs saved to: {full_logs_path}")
print(f"📊 File size: {full_logs_path.stat().st_size:,} bytes")
print(f"\n💡 Tip: These logs include:")
print("   - Complete prompts for each stage")
print("   - Full raw LLM responses (no truncation)")
print("   - Parsed payloads with validation results")
print("   - Complete message history")

### 📝 Why Truncate Logs?

**Memory & Performance Reasons:**

1. **Memory Bloat**: When processing 1,000+ synsets, full responses can consume GBs of RAM
2. **Serialization Size**: Saving results to JSON becomes impractical with full responses
3. **Quick Inspection**: Truncated logs allow fast debugging without scrolling through pages

**Best Practice:**
- Use `payload["logs"]` for quick debugging during development
- Use `payload["calls"]` to save full data for critical synsets
- Export full logs to separate files for detailed analysis (as shown above)

In [ ]:
# Using the log utilities for easier log management
from wordnet_autotranslate.utils.log_utils import save_full_logs, analyze_stage_lengths

# Save full logs with one function call
log_path = save_full_logs(result, output_path="output/full_logs_example.json")
print(f"✅ Saved to: {log_path}")

# Analyze response sizes per stage
print("\n📊 Response sizes by stage:")
lengths = analyze_stage_lengths(result)
for stage, length in sorted(lengths.items(), key=lambda x: x[1], reverse=True):
    print(f"  {stage:20} {length:>8,} chars")

## 🎯 New Feature: Per-Word Confidence Levels

The filtering stage now provides **confidence levels for each individual synonym**, allowing you to:
- Assess quality at the word level
- Filter results by confidence threshold
- Understand which synonyms are most reliable

In [ ]:
# Access per-word confidence levels from the filtering stage
filtering_payload = result["payload"]["filtering"]

print("=== PER-WORD CONFIDENCE LEVELS ===\n")

# Get the confidence dictionary
confidence_by_word = filtering_payload.get("confidence_by_word", {})

if confidence_by_word:
    print(f"Filtered synonyms with individual confidence levels:")
    for word, conf in confidence_by_word.items():
        emoji = "🟢" if conf == "high" else "🟡" if conf == "medium" else "🔴"
        print(f"  {emoji} {word:20} → {conf}")
    
    # Calculate statistics
    total = len(confidence_by_word)
    high = sum(1 for c in confidence_by_word.values() if c == "high")
    medium = sum(1 for c in confidence_by_word.values() if c == "medium")
    low = sum(1 for c in confidence_by_word.values() if c == "low")
    
    print(f"\n📊 Confidence Distribution:")
    print(f"  🟢 High:   {high}/{total} ({high/total*100:.1f}%)")
    print(f"  🟡 Medium: {medium}/{total} ({medium/total*100:.1f}%)")
    print(f"  🔴 Low:    {low}/{total} ({low/total*100:.1f}%)")
else:
    print("⚠️ No per-word confidence data available")
    print("(This field is optional - older pipeline versions may not include it)")

# Show overall filtering confidence
overall_confidence = filtering_payload.get("confidence", "N/A")
print(f"\n🎯 Overall filtering confidence: {overall_confidence}")

In [ ]:
# Practical use: Filter synonyms by confidence threshold
print("=== FILTERING BY CONFIDENCE THRESHOLD ===\n")

confidence_by_word = filtering_payload.get("confidence_by_word", {})

if confidence_by_word:
    # Get only high-confidence synonyms
    high_confidence = [word for word, conf in confidence_by_word.items() if conf == "high"]
    print(f"🟢 High-confidence only ({len(high_confidence)} words):")
    for word in high_confidence:
        print(f"  • {word}")
    
    # Get high + medium confidence
    reliable = [word for word, conf in confidence_by_word.items() if conf in ["high", "medium"]]
    print(f"\n🟢🟡 High + Medium confidence ({len(reliable)} words):")
    for word in reliable:
        conf = confidence_by_word[word]
        print(f"  • {word} ({conf})")
    
    print("\n💡 Use Case Examples:")
    print("  - Strict quality: Use only 'high' confidence words")
    print("  - Balanced: Use 'high' + 'medium' (default)")
    print("  - Exploratory: Include all levels for broader coverage")

In [ ]:
# View the improved filtering prompt that generated this result
print("=== IMPROVED FILTERING PROMPT ===\n")

filtering_call = result["payload"]["calls"]["filtering"]
prompt = filtering_call.get("prompt", "")

# Show key parts of the new prompt
print("Key improvements in the filtering prompt:")
print("\n1️⃣ Balanced Approach:")
print("   ✓ Preserve core concept while allowing natural variation")
print("   ✓ Prefer idiomatic expressions over literal translations")
print("\n2️⃣ Cultural Sensitivity:")
print("   ✓ Choose words typical in modern usage")
print("   ✓ Include culturally appropriate expressions")
print("\n3️⃣ Flexibility:")
print("   ✓ Allow abstract/concrete variants if natives use them")
print("   ✓ Reject only clearly different concepts")

print("\n" + "=" * 70)
print("Full Filtering Prompt:")
print("=" * 70)
print(prompt)

In [ ]:
# Examine what was removed during filtering and why
print("=== FILTERING DECISIONS ===\n")

removed_items = filtering_payload.get("removed", [])

if removed_items:
    print(f"❌ Removed candidates ({len(removed_items)} items):\n")
    for item in removed_items:
        word = item.get("word", "?")
        reason = item.get("reason", "No reason provided")
        print(f"  • {word:20} → {reason}")
    
    print("\n💡 These removals show the LLM's understanding of:")
    print("   - Which words are too broad/narrow")
    print("   - Which don't fit the cultural context")
    print("   - Which belong to different concepts")
else:
    print("✅ No candidates were removed!")
    print("   All expanded synonyms passed the filtering stage.")

# Show the progression
print("\n" + "=" * 70)
print("PIPELINE PROGRESSION")
print("=" * 70)
expanded = expansion_payload.get("expanded_synonyms", [])
filtered = filtering_payload.get("filtered_synonyms", [])

print(f"\n📊 Stage 4 - Expanded: {len(expanded)} candidates")
print(f"   {', '.join(expanded)}")
print(f"\n✅ Stage 5 - Filtered: {len(filtered)} final synonyms")
print(f"   {', '.join(filtered)}")
print(f"\n📉 Removal rate: {len(expanded) - len(filtered)}/{len(expanded)} candidates removed")

---

## 🔄 Additional Synset Translations

Let's translate four more synsets to demonstrate how the pipeline handles different types of words and semantic relationships.

### 🔹 Synset 2: "happiness" (abstract emotion)

In [ ]:
# Show next 4 synsets from the dataset and prepare them for translation
print("📋 Next 4 synsets available for translation:\n")

synsets_to_translate = []

for i in range(1, min(5, len(pairs))):
    pair = pairs[i]
    
    # Build synset structure from pair data (use 'lemmas' field for pipeline compatibility)
    synset = {
        "id": pair["english_id"],
        "pos": pair["english_pos"],
        "lexname": pair.get("english_domain", ""),
        "definition": pair["english_definition"],
        "examples": pair.get("english_examples", []),
        "lemmas": pair["english_lemmas"],  # Pipeline looks for 'lemmas' field
        "synonyms": pair["english_lemmas"],  # Keep for display
        "ili": "",  # Not in this dataset
        "topic_domains": []
    }
    
    synsets_to_translate.append(synset)
    
    print(f"{i+1}. {synset['id']} - {', '.join(synset['synonyms'][:2])}")
    print(f"   POS: {synset['pos']} | Domain: {synset.get('lexname', 'N/A')}")
    print(f"   Definition: {synset['definition'][:80]}...")
    print()

print(f"✅ Prepared {len(synsets_to_translate)} synsets for translation")

### 🔹 Synset 2: "condiment" (concrete noun)

In [ ]:
# Translate synset 2: condiment
synset_2 = synsets_to_translate[0]

print(f"🔄 Translating: {', '.join(synset_2['synonyms'])}")
print(f"   Definition: {synset_2['definition']}\n")

result_2 = pipeline.translate_synset(synset_2)

# Extract payloads (use correct field names)
filtering_payload_2 = result_2["payload"]["filtering"]
expansion_payload_2 = result_2["payload"]["expansion"]

print("\n✅ Translation complete!")
print(f"   Filtered: {', '.join(filtering_payload_2['filtered_synonyms'])}")
print(f"   Confidence: {filtering_payload_2['confidence']}")

In [ ]:
# Check Ollama status and model availability
import ollama

try:
    # Test if Ollama is responding
    test_response = ollama.chat(
        model=PREFERRED_OLLAMA_MODEL,
        messages=[{"role": "user", "content": "Respond with just: OK"}],
        options={"temperature": 0.0}
    )
    print(f"✅ Ollama is responding")
    print(f"   Model: {PREFERRED_OLLAMA_MODEL}")
    print(f"   Test response: {test_response['message']['content']}\n")
    
    # Try a simple JSON generation test
    json_test = ollama.chat(
        model=PREFERRED_OLLAMA_MODEL,
        messages=[{
            "role": "user",
            "content": "Return ONLY valid JSON (no markdown): {\"test\": \"value\"}"
        }],
        options={"temperature": 0.0}
    )
    print(f"JSON test response:")
    print(json_test['message']['content'][:200])
    
except Exception as e:
    print(f"❌ Ollama error: {e}")

In [ ]:
# Check all stages in result_2
print("=" * 70)
print("Checking all stages...")
print("=" * 70)

for stage_name, stage_data in result_2["payload"].items():
    if stage_name in ["calls", "logs"]:
        continue
    print(f"\n{stage_name}:")
    print(f"  Type: {type(stage_data)}")
    if isinstance(stage_data, dict):
        if "error" in stage_data:
            print(f"  ❌ Error: {stage_data['error']}")
        else:
            print(f"  Keys: {list(stage_data.keys())[:5]}")  # Show first 5 keys
    else:
        print(f"  Value: {str(stage_data)[:100]}")

In [ ]:
# Analysis for synset 2
print("=" * 70)
print("SYNSET 2 ANALYSIS: condiment")
print("=" * 70)

# Confidence breakdown
confidence_by_word_2 = filtering_payload_2.get("confidence_by_word", {})
if confidence_by_word_2:
    print("\n🎯 Per-word confidence:")
    for word, conf in confidence_by_word_2.items():
        emoji_map = {"high": "🟢", "medium": "🟡", "low": "🔴"}
        print(f"   {emoji_map.get(conf, '⚪')} {word:20} → {conf}")

# Removed items
removed_2 = filtering_payload_2.get("removed", [])
print(f"\n❌ Removed: {len(removed_2)} candidates")
if removed_2:
    for item in removed_2:
        print(f"   • {item.get('word', '?'):20} → {item.get('reason', 'No reason')}")

# Pipeline progression
expanded_2 = expansion_payload_2.get("expanded_synonyms", [])
filtered_2 = filtering_payload_2.get("filtered_synonyms", [])
print(f"\n📊 Expansion: {len(expanded_2)} → Filtering: {len(filtered_2)}")
print(f"   Before: {', '.join(expanded_2)}")
print(f"   After:  {', '.join(filtered_2)}")

In [ ]:
# Show definition translation for synset 2
print("=" * 70)
print("DEFINITION TRANSLATION")
print("=" * 70)

definition_payload_2 = result_2["payload"]["definition"]

print(f"\n🇬🇧 English definition:")
print(f"   {synset_2['definition']}\n")

print(f"🇷🇸 Serbian translation:")
print(f"   {definition_payload_2['definition_translation']}\n")

if definition_payload_2.get('notes'):
    print(f"📝 Translation notes:")
    print(f"   {definition_payload_2['notes']}\n")

# Compare with existing Serbian WordNet
print("=" * 70)
print("COMPARISON WITH EXISTING SERBIAN WORDNET")
print("=" * 70)

# Get the Serbian synset from the original pair
serbian_pair = pairs[1]  # Same index as synset_2

print(f"\n🆕 Our pipeline output:")
print(f"   Synonyms: {', '.join(filtering_payload_2['filtered_synonyms'])}")
print(f"   Confidence: {filtering_payload_2['confidence']}\n")

print(f"📚 Existing Serbian WordNet:")
print(f"   Synset ID: {serbian_pair['serbian_id']}")
print(f"   Synonyms: {', '.join(serbian_pair['serbian_synonyms'])}")
print(f"   Definition: {serbian_pair['serbian_definition']}\n")

# Find overlap
our_words = set(filtering_payload_2['filtered_synonyms'])
their_words = set(serbian_pair['serbian_synonyms'])

overlap = our_words & their_words
only_ours = our_words - their_words
only_theirs = their_words - our_words

print(f"🔄 Comparison:")
print(f"   ✅ Overlap: {', '.join(overlap) if overlap else 'None'}")
print(f"   🆕 Only in our output: {', '.join(only_ours) if only_ours else 'None'}")
print(f"   📚 Only in existing: {', '.join(only_theirs) if only_theirs else 'None'}")
print(f"   📊 Match rate: {len(overlap)}/{len(their_words)} ({len(overlap)/len(their_words)*100:.0f}% of existing synset)")

### 🔹 Synset 3: "scatter, sprinkle" (verb)

In [ ]:
# Translate synset 3: scatter, sprinkle
synset_3 = synsets_to_translate[1]

print(f"🔄 Translating: {', '.join(synset_3['synonyms'])}")
print(f"   Definition: {synset_3['definition']}\n")

result_3 = pipeline.translate_synset(synset_3)

filtering_payload_3 = result_3["payload"]["filtering"]
expansion_payload_3 = result_3["payload"]["expansion"]

print("\n✅ Translation complete!")
print(f"   Filtered: {', '.join(filtering_payload_3['filtered_synonyms'])}")
print(f"   Confidence: {filtering_payload_3['confidence']}")

In [ ]:
# Analysis for synset 3
print("=" * 70)
print("SYNSET 3 ANALYSIS: scatter, sprinkle")
print("=" * 70)

confidence_by_word_3 = filtering_payload_3.get("confidence_by_word", {})
if confidence_by_word_3:
    print("\n🎯 Per-word confidence:")
    for word, conf in confidence_by_word_3.items():
        emoji_map = {"high": "🟢", "medium": "🟡", "low": "🔴"}
        print(f"   {emoji_map.get(conf, '⚪')} {word:20} → {conf}")

removed_3 = filtering_payload_3.get("removed", [])
print(f"\n❌ Removed: {len(removed_3)} candidates")
if removed_3:
    for item in removed_3:
        print(f"   • {item.get('word', '?'):20} → {item.get('reason', 'No reason')}")

expanded_3 = expansion_payload_3.get("expanded_synonyms", [])
filtered_3 = filtering_payload_3.get("filtered_synonyms", [])
print(f"\n📊 Expansion: {len(expanded_3)} → Filtering: {len(filtered_3)}")
print(f"   Before: {', '.join(expanded_3)}")
print(f"   After:  {', '.join(filtered_3)}")

### 🔹 Synset 4: "pick, pluck" (verb)

In [ ]:
# Translate synset 4: pick, pluck
synset_4 = synsets_to_translate[2]

print(f"🔄 Translating: {', '.join(synset_4['synonyms'])}")
print(f"   Definition: {synset_4['definition']}\n")

result_4 = pipeline.translate_synset(synset_4)

filtering_payload_4 = result_4["payload"]["filtering"]
expansion_payload_4 = result_4["payload"]["expansion"]

print("\n✅ Translation complete!")
print(f"   Filtered: {', '.join(filtering_payload_4['filtered_synonyms'])}")
print(f"   Confidence: {filtering_payload_4['confidence']}")

In [ ]:
# Analysis for synset 4
print("=" * 70)
print("SYNSET 4 ANALYSIS: pick, pluck")
print("=" * 70)

confidence_by_word_4 = filtering_payload_4.get("confidence_by_word", {})
if confidence_by_word_4:
    print("\n🎯 Per-word confidence:")
    for word, conf in confidence_by_word_4.items():
        emoji_map = {"high": "🟢", "medium": "🟡", "low": "🔴"}
        print(f"   {emoji_map.get(conf, '⚪')} {word:20} → {conf}")

removed_4 = filtering_payload_4.get("removed", [])
print(f"\n❌ Removed: {len(removed_4)} candidates")
if removed_4:
    for item in removed_4:
        print(f"   • {item.get('word', '?'):20} → {item.get('reason', 'No reason')}")

expanded_4 = expansion_payload_4.get("expanded_synonyms", [])
filtered_4 = filtering_payload_4.get("filtered_synonyms", [])
print(f"\n📊 Expansion: {len(expanded_4)} → Filtering: {len(filtered_4)}")
print(f"   Before: {', '.join(expanded_4)}")
print(f"   After:  {', '.join(filtered_4)}")

### 🔹 Synset 5: "sweep" (verb)

In [ ]:
# Translate synset 5: sweep
synset_5 = synsets_to_translate[3]

print(f"🔄 Translating: {', '.join(synset_5['synonyms'])}")
print(f"   Definition: {synset_5['definition']}\n")

result_5 = pipeline.translate_synset(synset_5)

filtering_payload_5 = result_5["payload"]["filtering"]
expansion_payload_5 = result_5["payload"]["expansion"]

print("\n✅ Translation complete!")
print(f"   Filtered: {', '.join(filtering_payload_5['filtered_synonyms'])}")
print(f"   Confidence: {filtering_payload_5['confidence']}")

In [ ]:
# Analysis for synset 5
print("=" * 70)
print("SYNSET 5 ANALYSIS: sweep")
print("=" * 70)

confidence_by_word_5 = filtering_payload_5.get("confidence_by_word", {})
if confidence_by_word_5:
    print("\n🎯 Per-word confidence:")
    for word, conf in confidence_by_word_5.items():
        emoji_map = {"high": "🟢", "medium": "🟡", "low": "🔴"}
        print(f"   {emoji_map.get(conf, '⚪')} {word:20} → {conf}")

removed_5 = filtering_payload_5.get("removed", [])
print(f"\n❌ Removed: {len(removed_5)} candidates")
if removed_5:
    for item in removed_5:
        print(f"   • {item.get('word', '?'):20} → {item.get('reason', 'No reason')}")

expanded_5 = expansion_payload_5.get("expanded_synonyms", [])
filtered_5 = filtering_payload_5.get("filtered_synonyms", [])
print(f"\n📊 Expansion: {len(expanded_5)} → Filtering: {len(filtered_5)}")
print(f"   Before: {', '.join(expanded_5)}")
print(f"   After:  {', '.join(filtered_5)}")

---

## 📊 Comparative Summary

Compare the results across all 5 synsets to see how the pipeline handles different word types and semantic relationships.

In [ ]:
# Summary comparison of all 5 synsets
print("=" * 90)
print("COMPARATIVE SUMMARY: All 5 Synsets")
print("=" * 90)

results = [
    ("institution", synset_input, filtering_payload, expansion_payload),
    ("condiment", synset_2, filtering_payload_2, expansion_payload_2),
    ("scatter/sprinkle", synset_3, filtering_payload_3, expansion_payload_3),
    ("pick/pluck", synset_4, filtering_payload_4, expansion_payload_4),
    ("sweep", synset_5, filtering_payload_5, expansion_payload_5),
]

print(f"\n{'Synset':<18} {'POS':<5} {'Expanded':<10} {'Filtered':<10} {'Removed':<10} {'Confidence':<12}")
print("-" * 90)

for name, synset, filt_payload, exp_payload in results:
    pos = synset['pos']
    expanded_count = len(exp_payload.get('expanded_synonyms', []))
    filtered_count = len(filt_payload.get('filtered_synonyms', []))
    removed_count = len(filt_payload.get('removed', []))
    confidence = filt_payload.get('confidence', 'N/A')
    
    print(f"{name:<18} {pos:<5} {expanded_count:<10} {filtered_count:<10} {removed_count:<10} {confidence:<12}")

print("\n" + "=" * 90)
print("KEY INSIGHTS")
print("=" * 90)

# Calculate statistics
total_expanded = sum(len(r[3].get('expanded_synonyms', [])) for r in results)
total_filtered = sum(len(r[2].get('filtered_synonyms', [])) for r in results)
total_removed = sum(len(r[2].get('removed', [])) for r in results)

print(f"\n📈 Total candidates expanded: {total_expanded}")
print(f"✅ Total candidates filtered: {total_filtered}")
print(f"❌ Total candidates removed: {total_removed}")
print(f"📉 Average removal rate: {(total_removed/total_expanded*100):.1f}%")

# Confidence distribution
high_conf = sum(1 for r in results if r[2].get('confidence') == 'high')
medium_conf = sum(1 for r in results if r[2].get('confidence') == 'medium')
low_conf = sum(1 for r in results if r[2].get('confidence') == 'low')

print(f"\n🎯 Overall confidence distribution:")
print(f"   🟢 High: {high_conf}/5 synsets ({high_conf/5*100:.0f}%)")
print(f"   🟡 Medium: {medium_conf}/5 synsets ({medium_conf/5*100:.0f}%)")
print(f"   🔴 Low: {low_conf}/5 synsets ({low_conf/5*100:.0f}%)")

## 🎉 Pass 3 Key Findings - Calibrated Prompts

The calibrated filtering prompt successfully achieves our goals:

1. **Balanced Approach**: The pipeline accepts both strict synonyms and culturally appropriate variants
   - Example: **"ustanova" now KEPT** for "institution" (was removed in Pass 2)
   - Also includes: "sedište", "kancelarija", "zgrada" (culturally typical terms)
   
2. **Quality Filtering**: Still removes genuinely problematic translations
   - "centar" (too generic)
   - "institucija" (too abstract for physical building sense)
   - "začinski dodatak" (unnatural compound phrase)
   - "čistiti" (too general for "sweep")
   
3. **Per-Word Confidence**: Provides granular quality metrics
   - **100% high-confidence synsets** (5/5 synsets)
   - Individual word-level confidence for nuanced evaluation
   
4. **Optimal Removal Rate**: **37.7% removal rate** indicates:
   - Not too strict (preserving valid variants like "ustanova")
   - Not too lenient (filtering out poor matches and compounds)
   - Right in the "Goldilocks zone" between Pass 1 (39.6%) and Pass 2 (67.9%)
   
5. **Good Synset Size**: **Average 6.4 synonyms per synset**
   - Large enough to be useful for lexicographers
   - Matches Pass 1 quality while maintaining Pass 2's filtering rigor
   
6. **Aspect Handling**: Successfully captures both perfective and imperfective forms
   - "raspršiti/raspršivati", "posipati/posipavati", "metati/metnuti", "pometati/pometiti"

**🎯 Calibration Success**: The revised prompt achieves the balance we were seeking!

## 📖 Definition Translations

Let's examine how the pipeline translated the English definitions (glosses) into Serbian.

In [ ]:
# Show definition translations for all 5 synsets - comparing all three versions
results_with_synsets = [
    ("institution", synset_input, result, pairs[0]),
    ("condiment", synset_2, result_2, pairs[1]),
    ("scatter/sprinkle", synset_3, result_3, pairs[2]),
    ("pick/pluck", synset_4, result_4, pairs[3]),
    ("sweep", synset_5, result_5, pairs[4]),
]

for name, synset, res, serbian_pair in results_with_synsets:
    print("=" * 80)
    print(f"{name.upper()}")
    print("=" * 80)
    
    definition_payload = res["payload"]["definition"]
    
    print(f"\n🇬🇧 English (source):")
    print(f"   {synset['definition']}")
    
    print(f"\n🤖 Serbian (pipeline translation):")
    print(f"   {definition_payload['definition_translation']}")
    
    print(f"\n👤 Serbian (human translation from existing WordNet):")
    print(f"   {serbian_pair['serbian_definition']}")
    
    # Show how similar they are
    pipeline_def = definition_payload['definition_translation'].lower()
    human_def = serbian_pair['serbian_definition'].lower()
    
    if pipeline_def == human_def:
        print(f"\n   ✅ Identical translations!")
    elif pipeline_def in human_def or human_def in pipeline_def:
        print(f"\n   ⚠️ Partially overlapping translations")
    else:
        print(f"\n   ℹ️ Different translations (both may be valid)")
    
    if definition_payload.get('notes'):
        print(f"\n📝 Pipeline notes: {definition_payload['notes']}")
    
    print()

## 🔍 Comparison with Existing Serbian WordNet

Since our examples come from aligned Serbian-English synset pairs, we can compare our pipeline's output with the existing human-created Serbian WordNet synsets.

In [ ]:
# Compare our pipeline output with existing Serbian WordNet
comparison_data = [
    ("institution", result, filtering_payload, pairs[0]),
    ("condiment", result_2, filtering_payload_2, pairs[1]),
    ("scatter/sprinkle", result_3, filtering_payload_3, pairs[2]),
    ("pick/pluck", result_4, filtering_payload_4, pairs[3]),
    ("sweep", result_5, filtering_payload_5, pairs[4]),
]

print("=" * 90)
print("SYNSET COMPARISON: Pipeline vs Existing Serbian WordNet")
print("=" * 90)

total_overlap = 0
total_existing = 0
total_our = 0

for name, res, filt_payload, serbian_pair in comparison_data:
    print(f"\n{'=' * 90}")
    print(f"{name.upper()}")
    print(f"{'=' * 90}")
    
    # Our output
    our_words = set(filt_payload['filtered_synonyms'])
    our_confidence = filt_payload['confidence']
    
    # Existing Serbian WordNet
    their_words = set(serbian_pair['serbian_synonyms'])
    their_definition = serbian_pair['serbian_definition']
    
    # Calculate overlap
    overlap = our_words & their_words
    only_ours = our_words - their_words
    only_theirs = their_words - our_words
    
    print(f"\n🆕 Our pipeline ({len(our_words)} synonyms, confidence: {our_confidence}):")
    print(f"   {', '.join(sorted(our_words))}")
    
    print(f"\n📚 Existing Serbian WordNet ({len(their_words)} synonyms):")
    print(f"   {', '.join(sorted(their_words))}")
    print(f"   Definition: {their_definition}")
    
    print(f"\n🔄 Overlap analysis:")
    print(f"   ✅ Matches ({len(overlap)}): {', '.join(sorted(overlap)) if overlap else 'None'}")
    print(f"   🆕 Only in pipeline ({len(only_ours)}): {', '.join(sorted(only_ours)) if only_ours else 'None'}")
    print(f"   📚 Only in existing ({len(only_theirs)}): {', '.join(sorted(only_theirs)) if only_theirs else 'None'}")
    
    if len(their_words) > 0:
        match_rate = len(overlap) / len(their_words) * 100
        print(f"   📊 Match rate: {len(overlap)}/{len(their_words)} ({match_rate:.1f}%)")
    
    # Accumulate totals
    total_overlap += len(overlap)
    total_existing += len(their_words)
    total_our += len(our_words)

print(f"\n{'=' * 90}")
print("OVERALL STATISTICS")
print(f"{'=' * 90}")
print(f"\n📊 Total synonyms:")
print(f"   Our pipeline: {total_our}")
print(f"   Existing WordNet: {total_existing}")
print(f"   Matches: {total_overlap}")
print(f"   Overall match rate: {total_overlap}/{total_existing} ({total_overlap/total_existing*100:.1f}%)")

### 💡 Interpretation of Comparison Results

The comparison reveals several important insights:

**Expected Differences:**
- **Different valid translations**: Both our pipeline and human translators may choose different but equally valid synonyms
- **Aspectual variants**: Serbian verbs have perfective/imperfective forms - we may include both while existing synset has one
- **Granularity choices**: Our pipeline may be more or less specific in synonym selection

**Pipeline Advantages:**
- **Comprehensive coverage**: May find additional valid synonyms that humans didn't include
- **Consistency**: Follows systematic rules across all synsets
- **Per-word confidence**: Provides quality metrics for each synonym

**Human Advantages:**
- **Cultural nuance**: Native speakers may prefer certain expressions
- **Domain expertise**: May include domain-specific terms
- **Established usage**: Reflects actual WordNet community decisions

The goal is not 100% match rate, but rather to **complement** existing resources with high-quality, confidence-scored suggestions that lexicographers can review.

## 🔄 Iterative Expansion Feature

The pipeline now uses **iterative expansion** - running the expansion stage multiple times until no new synonyms appear (or reaching a maximum of 5 iterations). This ensures comprehensive synonym coverage since LLM outputs can vary between runs.

In [ ]:
# Show iterative expansion details for all 5 synsets
print("=" * 90)
print("ITERATIVE EXPANSION ANALYSIS")
print("=" * 90)

for name, synset, res, serbian_pair in results_with_synsets:
    expansion_payload = res["payload"]["expansion"]
    
    print(f"\n{name.upper()}")
    print("-" * 90)
    
    iterations_run = expansion_payload.get("iterations_run", 1)
    converged = expansion_payload.get("converged", False)
    synonym_provenance = expansion_payload.get("synonym_provenance", {})
    
    print(f"🔄 Iterations: {iterations_run}")
    print(f"✓ Converged: {'Yes' if converged else 'No (hit max limit)'}")
    print(f"📊 Total unique synonyms: {len(expansion_payload['expanded_synonyms'])}")
    
    # Count synonyms by iteration
    if synonym_provenance:
        iteration_counts = {}
        for syn, iter_num in synonym_provenance.items():
            iteration_counts[iter_num] = iteration_counts.get(iter_num, 0) + 1
        
        print(f"\n📈 Synonyms found per iteration:")
        for iter_num in sorted(iteration_counts.keys()):
            count = iteration_counts[iter_num]
            if iter_num == 0:
                print(f"   Initial: {count} synonyms")
            else:
                print(f"   Iteration {iter_num}: {count} new synonyms")

print("\n" + "=" * 90)
print("This iterative approach ensures comprehensive coverage while")
print("stopping early when the LLM has exhausted its synonym knowledge.")
print("=" * 90)
